In [ ]:
from structure_derivation.model.model import StructureDerivationModel, StructureDerivationModelConfig
import os
import torch
import librosa

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CHECKPOINTS_DIR = "/keshav/musical_structure_metrics/structure_derivation/artifacts/structure_derivation_model/checkpoint/"

config = StructureDerivationModelConfig()
model = StructureDerivationModel(config)
model.to(device)

ckpt = torch.load(os.path.join(CHECKPOINTS_DIR, "checkpoint.pt"), map_location=device)
if "module" in ckpt["model"]:
    model.module.load_state_dict(ckpt["model"]["module"])  # `.module` because of DDP
else:
    model.load_state_dict(ckpt["model"])  # `.module` because of DDP

audio_paths = [
    '/mnt/data/music_reward/musiccaps/data/JDWPJ1AiDKc.wav',
    '/mnt/data/music_reward/musiccaps/data/JDWPJ1AiDKc.wav'
]

# Example input
audio, sr = librosa.load(audio_paths[0], sr=32000, mono=True)

audio_tensor = torch.tensor(audio).unsqueeze(0).to(device)  # (1, T)
model.eval()
with torch.no_grad():
    output = model(audio_tensor, infer_mode=True)

print(output['latent_output'].shape)  # Example output key

/mnt/data/keshav/conda_envs/structure/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/data/keshav/conda_envs/structure/lib/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
import librosa
import os
import numpy as np
import torch
import torch.nn.functional as F
from structure_derivation.model.model import StructureDerivationModel, StructureDerivationModelConfig
from vendi_score import vendi

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

def load_model(ckpt_path, device):
    config = StructureDerivationModelConfig()
    model = StructureDerivationModel(config)
    model.to(device)

    ckpt = torch.load(ckpt_path, map_location=device)
    if "module" in ckpt["model"]:
        model.module.load_state_dict(ckpt["model"]["module"])  # DDP checkpoint
    else:
        model.load_state_dict(ckpt["model"])
    model.eval()
    return model

def split_audio(audio_path, segment_seconds=10, target_sr=32000):
    """Load audio and split into N non-overlapping segments of segment_seconds each."""
    audio, sr = librosa.load(audio_path, sr=target_sr, mono=True)
    segment_len = segment_seconds * target_sr
    total_len = len(audio)
    segments = []
    for start in range(0, total_len, segment_len):
        end = start + segment_len
        if end <= total_len:
            segments.append(audio[start:end])
    return segments, sr

def compute_embeddings(model, segments, device):
    """Pass each segment through the model to get latent_output embeddings."""
    embeddings = []
    for seg in segments:
        seg_tensor = torch.tensor(seg, dtype=torch.float32).unsqueeze(0).to(device)  # (1, T)
        with torch.no_grad():
            out = model(seg_tensor, infer_mode=True)
        embeddings.append(out["latent_output"])  # (1, D)
    return torch.cat(embeddings, dim=0)  # (N, D)

def compute_structure_derivation(embeddings):
    """Cosine similarity between first segment and all others."""
    ref = embeddings[0].unsqueeze(0)  # (1, D)
    sims = F.cosine_similarity(ref, embeddings[1:], dim=1)  # (N-1,)
    return sims.cpu().numpy()

def compute_vendi(embeddings):
    """Compute cosine similarity matrix between all embeddings."""
    norm_emb = F.normalize(embeddings, p=2, dim=1)  # (N, D)
    sim_matrix = torch.matmul(norm_emb, norm_emb.T)  # (N, N)
    sim_matrix = sim_matrix.cpu().numpy()
    vendi_score = vendi.score_K(sim_matrix)
    return vendi_score


# ----------------- Usage -----------------
CHECKPOINTS_DIR = "/keshav/musical_structure_metrics/structure_derivation/artifacts/structure_derivation_model/checkpoint/"
ckpt_path = os.path.join(CHECKPOINTS_DIR, "checkpoint.pt")

model = load_model(ckpt_path, device)

audio_path = '/mnt/data/marble/mtg_jamendo/mtg-jamendo-dataset/data/raw_30s_audio/91/1092591.mp3'
segments, sr = split_audio(audio_path, segment_seconds=10, target_sr=32000)
print(f"Split into {len(segments)} segments.")

embeddings = compute_embeddings(model, segments, device)
print("Embeddings shape:", embeddings.shape)  # (N, D)

similarities = compute_structure_derivation(embeddings)
print("Cosine similarities with S1:", similarities)

# Average Structure Derivation score
avg_structure_derivation = similarities.mean()
print("Average Structure Derivation similarity with S1:", avg_structure_derivation)

# Vendi score
vendi_score = compute_vendi(embeddings)
print("Vendi score:", vendi_score)


Split into 24 segments.
Embeddings shape: torch.Size([24, 768])
Cosine similarities with S1: [0.49996632 0.49005872 0.530892   0.39764658 0.41397092 0.4780046
 0.47594923 0.41005087 0.4875069  0.38399807 0.4577372  0.48997647
 0.4386139  0.45407566 0.45616674 0.4551477  0.4112865  0.48017648
 0.44371143 0.48575637 0.57440317 0.6562359  0.6762432 ]
Average Structure Derivation similarity with S1: 0.4803293
Vendi score: 2.595095


In [1]:
1+1

2